In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [70]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [71]:
## Drop irrelavant columns
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)



In [72]:
## Encode categorical variable
label_encoder_gender= LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [73]:

## Onehot encode Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])

geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

## Combine one hot encoder columns with the original data 
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [74]:
## Divide the dataset into independent and dependent features
x = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']



In [75]:
## split the data in traning and testing sets
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

## Scale these features
scaler = StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [76]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo','wb') as file:
    pickle.dump(onehot_encoder_geo,file)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

Build our Model For ANN Regrassion statement


In [77]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [78]:
### Build Our Model
model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), #HL2
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid')# output layer


])

# compile the model
# Note: loss are different in classification problem(eg.binary_crossentropy) and regression pronlem statement(eg.mean_absolute_error)

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [79]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [80]:
## set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


log_dir= "log/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir= log_dir,histogram_freq=1)

In [81]:
## set up Early Stopping
early_stopping_callback= EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


In [82]:
## Train the model
history= model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs= 100,
    callbacks=[tensorflow_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 9ms/step - loss: 100430.2891 - mae: 100430.2891 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 100430.2969 - mae: 100430.2969 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 100430.3047 - mae: 100430.3047 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 100430.3125 - mae: 100430.3125 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 100430.3203 - mae: 100430.3203 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 100430.3359 - mae: 100430.3359 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 100430.2578 - mae

In [88]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [101]:

# uncomment to run tensorboard
%tensorboard --logdir log/fit20250725-144008


Reusing TensorBoard on port 6006 (pid 14832), started 0:03:32 ago. (Use '!kill 14832' to kill it.)

In [102]:
## Evalute model on the test data
test_loss,test_mae =model.evaluate(x_test,y_test)
print(f'Test MAE:{test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 98725.0625 - mae: 98725.0625
Test MAE:98725.0625


In [103]:
model.save('regression_model.h5')

c:\Users\viv96\OneDrive\Desktop\AL&ML Project\ANN End-to-End Project\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
